### Import Basic

In [12]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(os.path.join('..')))

### Compile `libc`

In [13]:
import subprocess

try:
    result = subprocess.run(
        "make -C ../libc clean && make -C ../libc all",
        shell=True,
        capture_output=True,
        text=True
    )
    print(result.stdout)

    if result.stderr:
        print(result.stderr)
    
    if result.returncode != 0:
        print(f"Build failed with exit code {result.returncode}")
        sys.exit(1)

except Exception as e:
    print(f"Build failed: {e}")
    sys.exit(1)

make: Entering directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/libc'
make: Leaving directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/libc'
make: Entering directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/libc'
OS        : Windows
Arch      : AMD64
Extension : .dll
Compiler  : gcc
  CC      src/array.c
  CC      src/linearModel.c
  CC      src/math.c
  CC      src/utils.c
  LINK    build/libc.dll
make: Leaving directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/libc'



### Import Loader

In [14]:
from engine.interop.loader import Loader

Loader._instance = None

try:
    Loader(
        lib_name="libc",
        lib_folder="../libc",
        build_folder="../libc/build",
        specs_folder="../libc/specs"
    )

except Exception as e:
    print(f"Error: {e}")
    raise SystemExit(1)

### Import Interop

In [15]:
import engine.interop.math as Math
from engine.interop.array import ArrayFloat32Ptr

### Math Tests

In [16]:
assert Math.addition(10, 5) == 15.0, f"Expected 15.0, got {Math.addition(10, 5)}"
assert Math.subtraction(10, 5) == 5.0, f"Expected 5.0, got {Math.subtraction(10, 5)}"
assert Math.multiplication(10, 5) == 50.0, f"Expected 50.0, got {Math.multiplication(10, 5)}"
assert Math.division(10, 2) == 5.0, f"Expected 5.0, got {Math.division(10, 2)}"
assert Math.power(2, 3) == 8.0, f"Expected 8.0, got {Math.power(2, 3)}"

try:
    Math.division(10, 0)
    raise AssertionError("Expected RuntimeError for division by zero")

except RuntimeError as e:
    print(f"Error 2/2: {e}")

Error 2/2: Math.division(): _call(): Loader.call(my_div): Loader.check_status(): Division by Zero


### ArrayFloat32Ptr Tests

In [17]:
# Test 1: init_from_incrementing_numbers
try:
    expected = [0.0, 1.0, 2.0, 3.0, 4.0]
    arr = ArrayFloat32Ptr.init_from_incrementing_numbers(5)
    assert arr.array == expected, f"Expected {expected}, got {arr.array}"

except Exception as e:
    print(f"❌ FAIL: {e}")

In [18]:
# Test 2: Création avec liste
try:
    data = [1.5, 2.5, 3.5]
    arr = ArrayFloat32Ptr(data)
    assert arr.array == data, f"Expected {data}, got {arr.array}"

except Exception as e:
    print(f"❌ FAIL: {e}")

In [19]:
# Test 3: sum()
try:
    arr = ArrayFloat32Ptr([1.0, 2.0, 3.0, 4.0])
    result = arr.sum()
    expected = 10.0
    assert result == expected, f"Expected {expected}, got {result}"

except Exception as e:
    print(f"❌ FAIL: {e}")

In [20]:
# Test 4: Accès et modification via array property
try:
    arr = ArrayFloat32Ptr([1.0, 2.0, 3.0])
    assert arr.array[1] == 2.0, f"Expected 2.0, got {arr.array[1]}"
    
    # Modification via reassignation complète
    new_data = arr.array
    new_data[1] = 5.0
    arr.array = new_data
    
    assert arr.array[1] == 5.0, f"Expected 5.0, got {arr.array[1]}"

except Exception as e:
    print(f"❌ FAIL: {e}")

In [21]:
# Test 5: array property retourne la liste
try:
    arr = ArrayFloat32Ptr([1.0, 2.0, 3.0])
    result = arr.array
    expected = [1.0, 2.0, 3.0]
    assert result == expected, f"Expected {expected}, got {result}"
    
except Exception as e:
    print(f"❌ FAIL: {e}")

In [22]:
# Test 6: Modification via property array
try:
    arr = ArrayFloat32Ptr([1.0, 2.0])
    arr.array = [10.0, 20.0, 30.0]
    assert arr.array == [10.0, 20.0, 30.0], f"Expected [10.0, 20.0, 30.0], got {arr.array}"
    assert arr.sum() == 60.0, f"Expected 60.0, got {arr.sum()}"

except Exception as e:
    print(f"❌ FAIL: {e}")

In [23]:
# Test 7: Validation liste vide
try:
    arr = ArrayFloat32Ptr([])
    print("❌ FAIL: Should have raised TypeError")

except TypeError as e:
    print(f"✅ TypeError Caught: {e}")

except Exception as e:
    print(f"❌ FAIL: Wrong error type: {e}")

✅ TypeError Caught: ArrayFloat32Ptr.__init__(): `data` must be a non-empty list of floats


In [24]:
# Test 8: Validation types incorrects
try:
    arr = ArrayFloat32Ptr([1.0, "test", 3.0])
    print("❌ FAIL: Should have raised TypeError")

except TypeError as e:
    print(f"✅ TypeError Caught: {e}")

except Exception as e:
    print(f"❌ FAIL: Wrong error type: {e}")

✅ TypeError Caught: ArrayFloat32Ptr.__init__(): all elements of the list must be floats


In [25]:
# Test 9: Index hors limites
try:
    arr = ArrayFloat32Ptr([1.0, 2.0])
    val = arr.array[10]
    print("❌ FAIL: Should have raised IndexError")

except IndexError as e:
    print(f"✅ Error Caught: {e}")

except Exception as e:
    print(f"❌ FAIL: Wrong error type: {e}")

✅ Error Caught: list index out of range


In [26]:
# Test 10: Gros tableau (performance)
try:
    arr = ArrayFloat32Ptr.init_from_incrementing_numbers(1000)
    result = arr.sum()
    expected = sum(range(1000))  # 0+1+2+...+999
    assert result == expected, f"Expected {expected}, got {result}"
    
except Exception as e:
    print(f"❌ FAIL: {e}")
